# Node2Vec

Não deu muito certo. O treinamento demora bastante

In [66]:
import os

import pandas as pd
import networkx as nx
import numpy as np
import seaborn as sns

from fastdtw import fastdtw
from tqdm import tqdm
from node2vec import Node2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.spatial.distance import euclidean

In [2]:
def batch_read(path: str) -> list[pd.DataFrame]:
    df_list = []
    for file in tqdm(os.listdir(path)):
        df = pd.read_csv(f'{path}/{file}')
        df_list.append(df)
    return df_list

In [3]:
control_data = batch_read('../data/Controls_columns')

100%|██████████| 66/66 [00:03<00:00, 17.74it/s]


In [4]:
parkinson_data = batch_read('../data/PDs_columns')

100%|██████████| 113/113 [00:06<00:00, 18.38it/s]


In [5]:
np.unique([column.split('.')[0] for column in parkinson_data[0].columns])

array(['AAL3', 'CSF', 'Grey Matter', 'Shen_268', 'White Matter', 'atlas',
       'networks'], dtype='<U12')

Os atlas possíveis são AAL3, Shen_268 e atlas. Cada atlas particiona o cérebro de maneiras diferentes. Devemos selecionar as colunas correspondentes para aplicar o processamento

In [6]:
all_columns = parkinson_data[0].columns
AAL3_columns = all_columns[np.where([column.split('.')[0] == 'AAL3' for column in all_columns])[0]]
AAL3_columns[:5]

Index(['AAL3.cluster001', 'AAL3.cluster002', 'AAL3.cluster003',
       'AAL3.cluster004', 'AAL3.cluster005'],
      dtype='object')

In [ ]:
control_AAL3_data = [data[AAL3_columns] for data in control_data]
parkinson_AAL3_data = [data[AAL3_columns] for data in parkinson_data]

## Create Graphs

Demora muito para calcular a rede funcional com DTW, portanto vamos utilizar Pearson

In [12]:
parkinson_AAL3_data[0].shape

(240, 166)

In [78]:
AAL3_data = np.concatenate([
    parkinson_AAL3_data,
    control_AAL3_data
], axis=0)

In [94]:
y = np.concatenate([
    [1 for _ in range(len(parkinson_data))],
    [0 for _ in range(len(control_data))]
])

In [96]:
print(AAL3_data.shape, y.shape)

(179, 240, 166) (179,)


In [91]:
def get_connectivity_graph(time_series: np.array, threshold: float = 0.15) -> nx.Graph:
    if threshold <= 0 or threshold > 1:
        raise ValueError(f'Threshold should be a value between 0 and 1. Got {threshold}')
    
    correlation_matrix = np.corrcoef(time_series)
    adjacency_matrix = correlation_matrix - np.eye(len(correlation_matrix)) # remove selfloops

    adjacency_matrix[np.abs(adjacency_matrix) > threshold] = 1
    adjacency_matrix[np.abs(adjacency_matrix) <= threshold] = 0

    connectivity_graph = nx.from_numpy_array(adjacency_matrix)

    return connectivity_graph

In [92]:
connectivity_graphs = [get_connectivity_graph(time_series) for time_series in tqdm(AAL3_data)]

100%|██████████| 179/179 [00:16<00:00, 10.80it/s]


### Node2Vec

Vamos utilizar como uma ferramenta para Embeddar os grafos

In [104]:
def get_graph_embeddings(graph: nx.Graph) -> np.array:
    node2vec = Node2Vec(
        graph, 
        dimensions=5,
        walk_length=10,
        num_walks=10,
        workers=2,
        quiet=True
    ).fit(
        window=10,
        min_count=1,
        batch_words=4
    )

    embeddings = node2vec.wv.vectors

    return embeddings

In [106]:
graph_embeddings = [get_graph_embeddings(graph) for graph in tqdm(connectivity_graphs)]



































































































  1%|          | 2/179 [08:26<12:27:14, 253.30s/it]






































































































































































































































































100%|██████████| 179/179 [26:50<00:00,  9.00s/it]


In [113]:
for i, embedding in enumerate(graph_embeddings):
    df = pd.DataFrame(embedding)
    df.to_csv(f'../data/graph_embeddings/embedding_{i}.csv')

#### Train/Test Split

In [122]:
flatten_graph_embeddings = [embedding.flatten() for embedding in graph_embeddings]

In [135]:
X_train, X_test, y_train, y_test = train_test_split(flatten_graph_embeddings, y, shuffle=True)

#### Training

In [164]:
grid_params = {
    'alpha': np.arange(1e-3, 1e-1, 1e-2),
    'learning_rate': ['constant', 'adaptive'],
}

fixed_parameters = {
    'early_stopping': True,
    'hidden_layer_sizes': (100),
    'activation': 'tanh',
    'learning_rate_init': 0.00001
}

In [165]:
model = MLPClassifier(**fixed_parameters)
hyperparam_optimization = RandomizedSearchCV(model, params, random_state=1)
search = hyperparam_optimization.fit(X_train, y_train)
search.best_params_

{'learning_rate': 'constant', 'alpha': 0.030999999999999996}

In [166]:
optimized_model = MLPClassifier(**fixed_parameters, **search.best_params_).fit(X_train, y_train)
optimized_model.score(X_test, y_test)

0.5777777777777777

In [168]:
optimized_model.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1])

Not good results with Node2Vec